In [ ]:
import param as pm
import panel as pn
import pandas as pd
import hvplot.pandas
import holoviews as hv
import numpy as np
import numbers
import curvesim
from curvesim.pool.stableswap import CurvePool as CurvesimPool
pn.extension()

In [ ]:
class StableSwap(pm.Parameterized):
    n = pm.Integer(3, doc="Number of coins.")
    balances = pm.List(default=[10]*3, item_type=numbers.Number)
    leverage = pm.Integer(1, bounds=(0,100), doc="Chi leverage parameter in whitepaper")
    D = pm.Number(1, softbounds=(0, 50), bounds=(0, None), doc="Invariant constant to be discovered.")
    S = pm.Number(constant=True)
    P = pm.Number(constant=True)
    
    def __init__(self, **params):
        super().__init__(**params)
        with pm.edit_constant(self):
            self.S = np.sum(self.balances)
            self.P = np.prod(self.balances)
        self.update_balances()
        
    @pm.depends('balances', watch=True)
    def update_balances(self):
        with pm.edit_constant(self):
            self.S = np.sum(self.balances)
            self.P = np.prod(self.balances)
            
    def L(self, D):
        """Leveraged Sum Factor"""
        return self.leverage * D ** (self.n-1)
    
    def K(self, D):
        """Constant Product"""
        return (D / self.n) ** self.n
        
    def left_invariant(self, D):
        """Leveraged Sum plus Product"""
        return self.L(D) * self.S + self.P
    
    def right_invariant(self, D):
        """Leveraged Constant plus Constant Product"""
        return  self.L(D) * D +  self.K(D)
    
    def invariant_curves(self):
        Ds = np.linspace(*s.param['D'].softbounds, num=10_000)
        ls = self.left_invariant(Ds)
        rs = self.right_invariant(Ds)
        df = pd.DataFrame({'D':Ds,'ls':ls,'rs':rs, 'Invariant': np.abs(ls-rs)})
        return df
    
    def minimum_invariant(self):
        df = self.invariant_curves()
        return df.iloc[[df['Invariant'].idxmin()]].T
    
    @pm.depends('n', 'balances', 'leverage')
    def view_invariant_curves(self):
        invariant_curves = self.invariant_curves()
        invariant_curves_chart = invariant_curves.hvplot(x='D', y=['ls', 'rs', 'Invariant'])
        invariant_curves_chart.opts(title='Invariant Curves')
        return invariant_curves_chart
    
    def view_D(self):
        D = hv.VLine(self.D).opts(color='black')
        return D
    
    def view_chart(self):
        chart = self.view_invariant_curves() * self.view_D()
        return chart
    
    def view(self):
        return pn.Row(self, pn.Column(self.view_chart, self.minimum_invariant))

In [ ]:
s = StableSwap()
s.view()

Row
    [0] Column(margin=5, name='StableSwap', width=300)
        [0] StaticText(value='<b>StableSwap</b>')
        [1] IntInput(name='N', value=3)
        [2] ListInput(name='Balances', type=<class 'list'>, value=[10, 10, 10])
        [3] IntSlider(end=100, name='Leverage', value=1)
        [4] FloatSlider(end=50, name='D', value=1)
        [5] FloatInput(disabled=True, name='S', value=30)
        [6] FloatInput(disabled=True, name='P', value=1000)
    [1] Column
        [0] ParamMethod(method, _pane=HoloViews)
        [1] ParamMethod(method, _pane=DataFrame)

In [ ]:
class StableSwap(pm.Parameterized):
    n = pm.Integer(constant=True, doc="Number of coins.")
    balances = pm.List(default=[20, 30, 40], item_type=numbers.Number)
    A = pm.Number(1, softbounds=(0, 100), bounds=(0, None))
    D = pm.Number(1, softbounds=(1, 100), bounds=(1, None), doc="Invariant constant to be discovered.")
    leverage = pm.Number(constant=True, softbounds=(0,100), bounds=(0, None), doc="Chi leverage parameter in whitepaper")
    S = pm.Number(constant=True)
    P = pm.Number(constant=True)
    set_D = pm.Action(lambda self: self._set_D())
    
    
    def __init__(self, **params):
        super().__init__(**params)
        self._set_D()
        self.update_balances()
        
    @pm.depends('A', 'balances', watch=True)
    def _set_D(self):
        self.update_balances()
        self.D = CurvesimPool(A=self.A, D=self.balances, n=self.n).D()
        self.param['D'].softbounds = [self.D/4+1, self.D*2]
        self.update_leverage()

        
    @pm.depends()
    def update_balances(self):
        with pm.edit_constant(self):
            self.n = len(self.balances)
            self.S = np.sum(self.balances)
            self.P = np.prod(self.balances)
            
    @pm.depends()
    def update_leverage(self):
        with pm.edit_constant(self):
            self.leverage = self.get_leverage(self.D)
            
    def get_leverage(self, D):
        return self.A * self.P / self.K(D)
            
    def L(self, D):
        """Leveraged Sum Factor"""
        return self.get_leverage(D) * D ** (self.n-1)
    
    def K(self, D):
        """Constant Product"""
        return (D / self.n) ** self.n
        
    def left_invariant(self, D):
        """Leveraged Sum plus Product"""
        return self.L(D) * self.S + self.P
    
    def right_invariant(self, D):
        """Leveraged Constant plus Constant Product"""
        return  self.L(D) * D +  self.K(D)
    
    def invariant_curves(self):
        Ds = np.linspace(*s.param['D'].softbounds, num=1000)
        ls = self.left_invariant(Ds)
        rs = self.right_invariant(Ds)
        df = pd.DataFrame({'D':Ds,'ls':ls,'rs':rs, 'Invariant': np.abs(ls-rs)})
        return df
    
    def minimum_invariant(self):
        df = self.invariant_curves()
        return df.iloc[[df['Invariant'].idxmin()]].T
    
    @pm.depends('n', 'balances', 'leverage')
    def view_invariant_curves(self):
        invariant_curves = self.invariant_curves()
        invariant_curves_chart = invariant_curves.hvplot.area(x='D', y=['ls', 'rs', 'Invariant'])
        invariant_curves_chart.opts(title='Invariant Curves')
        return invariant_curves_chart
    
    def view_D(self):
        D = hv.VLine(self.D).opts(color='black')
        return D
    
    def view_chart(self):
        chart = self.view_invariant_curves() * self.view_D()
        return chart
    
    def view(self):
        return pn.Row(self, self.view_chart)

In [ ]:
s = StableSwap()
s.view()

Row
    [0] Column(margin=5, name='StableSwap', width=300)
        [0] StaticText(value='<b>StableSwap</b>')
        [1] IntInput(disabled=True, name='N', value=3)
        [2] ListInput(name='Balances', type=<class 'list'>, value=[20, 30, 40])
        [3] FloatSlider(end=100, name='A', value=1)
        [4] FloatSlider(end=176, name='D', start=23.0, value=88)
        [5] FloatSlider(disabled=True, end=100, name='Leverage', value=0.9508827948910594)
        [6] FloatInput(disabled=True, name='S', value=90)
        [7] FloatInput(disabled=True, name='P', value=24000)
        [8] Button(name='Set D')
    [1] ParamMethod(method, _pane=HoloViews)

In [ ]:

p = [10**18, 10**30, 10**30]
balances = [
    10**30,
    10**18,
    10**18,
]
virtual_balances = [b * p // 10**18 for b, p in zip(balances, p)]
N_COINS = 3
A = 2000

curvesim_pool = CurvesimPool(A=A, D=[10,1,1], n=N_COINS)

curvesim_pool.D()

In [ ]:
class CurvePool(pm.Parameterized):
    A = pm.Integer(85, doc="Amplification Coefficient.")
    D = pm.Integer(doc="Virtual total balance.")
    n = pm.Integer(3, doc="Number of coins.")
    balances = pm.List(default=[10_000]*3, item_type=numbers.Number)
    fee = pm.Number(0.0006)
    
    def get_D(self):
        D = self.A * self.n ** self.n * sum(self.balances)
    
    def view(self):
        return pn.Row(self)

In [ ]:
pool = CurvePool()

pool.view()

Row
    [0] Column(margin=5, name='CurvePool', width=300)
        [0] StaticText(value='<b>CurvePool</b>')
        [1] IntInput(name='A', value=85)
        [2] IntInput(name='D')
        [3] IntInput(name='N', value=3)
        [4] ListInput(name='Balances', type=<class 'list'>, value=[10000, 10000, 10000])
        [5] FloatInput(name='Fee', value=0.0006)

In [ ]:
def mainnet_3pool_state():
    """Snapshot of Mainnet 3Pool values"""
    p = [10**18, 10**30, 10**30]
    balances = [
        295949605740077243186725223,
        284320067518878,
        288200854907854,
    ]
    virtual_balances = [b * p // 10**18 for b, p in zip(balances, p)]

    return {
        "N_COINS": 3,
        "A": 2000,
        "p": p,
        "balances": balances,
        "virtual_balances": virtual_balances,
        "lp_tokens": 849743149250065202008212976,
        "virtual_price": 1022038799187029697,
    }

In [ ]:
def _vyper_3pool(mainnet_3pool_state):
    """Initialize vyper fixture using mainnet values."""
    lp_total_supply = mainnet_3pool_state["lp_tokens"]
    mock_filepath = os.path.join(_base_dir, "lp_token_mock.vy")
    lp_token = boa.load(mock_filepath, lp_total_supply)

    pool_filepath = os.path.join(_curve_dir, "basepool.vy")
    owner = FAKE_ADDRESS
    coins = [FAKE_ADDRESS] * 3
    A = mainnet_3pool_state["A"]
    fee = 4 * 10**6
    admin_fee = 5 * 10**9
    pool = boa.load(pool_filepath, owner, coins, lp_token, A, fee, admin_fee)

    balances = mainnet_3pool_state["balances"]
    pool.eval(f"self.balances={balances}")

    return pool